# AutoSentry AI - Predictive Vehicle Maintenance Model Training

**Team:** EverLearners  
**Project:** EY Techathon 6.0 - Autonomous Predictive Vehicle Maintenance System

This notebook trains machine learning models to predict vehicle maintenance failures using synthetic telematics data and historical maintenance records.

## Objectives:
1. Load and preprocess vehicle telemetry data
2. Engineer predictive features from sensor readings
3. Train RandomForest and XGBoost classifiers
4. Evaluate model performance and select best model
5. Save model artifacts for production deployment

## 1. Import Required Libraries

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import json
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    mean_absolute_error, mean_squared_error
)

# XGBoost (optional - install with: pip install xgboost)
try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("XGBoost not installed. Using GradientBoosting as alternative.")

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model persistence
import joblib

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"XGBoost available: {XGBOOST_AVAILABLE}")

## 2. Load and Explore Synthetic Vehicle Data

We load telemetry data from our 10 synthetic vehicles. Each vehicle has sensor readings including:
- Engine temperature (°C)
- Oil pressure (PSI)
- Battery voltage (V)
- Tire pressure (PSI)
- Brake pad wear (%)
- Coolant level (%)
- Transmission temperature (°C)

In [ ]:
# Data paths
DATA_DIR = "../../data"
TRAINING_DATA_PATH = os.path.join(DATA_DIR, "training_data.csv")
TELEMETRY_PATH = os.path.join(DATA_DIR, "vehicle_telemetry.json")
MAINTENANCE_PATH = os.path.join(DATA_DIR, "maintenance_records.json")

# Load training data
if os.path.exists(TRAINING_DATA_PATH):
    df = pd.read_csv(TRAINING_DATA_PATH)
    print(f"Loaded training data: {df.shape[0]} samples, {df.shape[1]} features")
else:
    # Generate synthetic data if not available
    print("Generating synthetic training data...")
    np.random.seed(42)
    n_samples = 1000
    
    df = pd.DataFrame({
        'vehicle_id': np.random.choice([f'VH{i:03d}' for i in range(1, 11)], n_samples),
        'timestamp': pd.date_range('2025-01-01', periods=n_samples, freq='6H'),
        'engine_temp': np.random.normal(95, 15, n_samples),
        'oil_pressure': np.random.normal(40, 8, n_samples),
        'battery_voltage': np.random.normal(12.3, 0.8, n_samples),
        'fuel_level': np.random.uniform(10, 100, n_samples),
        'tire_pressure_avg': np.random.normal(31, 2, n_samples),
        'brake_pad_wear_avg': np.random.uniform(0.1, 0.9, n_samples),
        'odometer': np.random.uniform(10000, 180000, n_samples),
        'speed': np.random.uniform(0, 80, n_samples),
        'rpm': np.random.normal(2500, 500, n_samples),
        'coolant_level': np.random.uniform(0.4, 1.0, n_samples),
        'transmission_temp': np.random.normal(90, 20, n_samples),
        'check_engine_light': np.random.choice([0, 1], n_samples, p=[0.85, 0.15])
    })
    
    # Create target variable based on conditions
    df['failure_within_30_days'] = 0
    df.loc[df['engine_temp'] > 110, 'failure_within_30_days'] = 1
    df.loc[df['oil_pressure'] < 28, 'failure_within_30_days'] = 1
    df.loc[df['battery_voltage'] < 11, 'failure_within_30_days'] = 1
    df.loc[df['brake_pad_wear_avg'] < 0.15, 'failure_within_30_days'] = 1
    df.loc[df['coolant_level'] < 0.5, 'failure_within_30_days'] = 1
    df.loc[df['transmission_temp'] > 120, 'failure_within_30_days'] = 1
    
    # Create failure type
    df['failure_type'] = 'none'
    df.loc[df['engine_temp'] > 110, 'failure_type'] = 'cooling_system'
    df.loc[df['oil_pressure'] < 28, 'failure_type'] = 'engine'
    df.loc[df['battery_voltage'] < 11, 'failure_type'] = 'electrical'
    df.loc[df['brake_pad_wear_avg'] < 0.15, 'failure_type'] = 'brakes'
    df.loc[(df['coolant_level'] < 0.5) | (df['transmission_temp'] > 120), 'failure_type'] = 'multiple'

# Display data info
print("\nDataset Shape:", df.shape)
print("\nColumn Data Types:")
print(df.dtypes)
print("\nFirst 5 Rows:")
df.head()

In [ ]:
# Statistical summary
print("Statistical Summary:")
df.describe()

## 3. Data Preprocessing and Feature Engineering

Create additional features from raw sensor data:
- Health scores for each system
- Rolling averages for trend detection
- Rate of change features
- Normalized mileage features

In [ ]:
# Feature Engineering
def engineer_features(df):
    """Create derived features for better prediction"""
    df_fe = df.copy()
    
    # Engine health score (normalized, 1 = optimal, 0 = critical)
    df_fe['engine_health'] = 1.0 - np.clip((df_fe['engine_temp'] - 90) / 40, 0, 1)
    
    # Oil health score
    df_fe['oil_health'] = np.clip(df_fe['oil_pressure'] / 45, 0, 1)
    
    # Battery health score
    df_fe['battery_health'] = np.clip((df_fe['battery_voltage'] - 10) / 3, 0, 1)
    
    # Cooling system health
    df_fe['cooling_health'] = np.clip(df_fe['coolant_level'], 0, 1)
    
    # Transmission health
    df_fe['trans_health'] = 1.0 - np.clip((df_fe['transmission_temp'] - 80) / 50, 0, 1)
    
    # Overall health score (weighted average)
    df_fe['overall_health'] = (
        0.25 * df_fe['engine_health'] +
        0.20 * df_fe['oil_health'] +
        0.15 * df_fe['battery_health'] +
        0.20 * df_fe['cooling_health'] +
        0.20 * df_fe['trans_health']
    )
    
    # Mileage tier (normalized)
    df_fe['mileage_normalized'] = df_fe['odometer'] / 200000
    
    # High mileage flag
    df_fe['high_mileage'] = (df_fe['odometer'] > 100000).astype(int)
    
    # Critical indicators
    df_fe['critical_temp'] = (df_fe['engine_temp'] > 105).astype(int)
    df_fe['low_oil'] = (df_fe['oil_pressure'] < 30).astype(int)
    df_fe['low_battery'] = (df_fe['battery_voltage'] < 11.5).astype(int)
    df_fe['low_brakes'] = (df_fe['brake_pad_wear_avg'] < 0.2).astype(int)
    
    # Risk score (sum of critical indicators)
    df_fe['risk_score'] = (
        df_fe['critical_temp'] + 
        df_fe['low_oil'] + 
        df_fe['low_battery'] + 
        df_fe['low_brakes'] +
        df_fe['check_engine_light']
    )
    
    return df_fe

# Apply feature engineering
df_processed = engineer_features(df)
print(f"Features after engineering: {df_processed.shape[1]}")
print("\nNew features created:")
new_features = ['engine_health', 'oil_health', 'battery_health', 'cooling_health', 
                'trans_health', 'overall_health', 'mileage_normalized', 'high_mileage',
                'critical_temp', 'low_oil', 'low_battery', 'low_brakes', 'risk_score']
df_processed[new_features].head()

## 4. Exploratory Data Analysis and Visualizations

In [ ]:
# Target variable distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Class distribution
failure_counts = df_processed['failure_within_30_days'].value_counts()
axes[0].bar(['No Failure', 'Failure'], failure_counts.values, color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Target Variable Distribution', fontsize=14)
axes[0].set_ylabel('Count')
for i, v in enumerate(failure_counts.values):
    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Failure rate by vehicle
if 'vehicle_id' in df_processed.columns:
    vehicle_failure_rate = df_processed.groupby('vehicle_id')['failure_within_30_days'].mean().sort_values(ascending=False)
    axes[1].barh(vehicle_failure_rate.index, vehicle_failure_rate.values, color='#3498db')
    axes[1].set_xlabel('Failure Rate')
    axes[1].set_title('Failure Rate by Vehicle', fontsize=14)

plt.tight_layout()
plt.show()

print(f"\nClass Distribution:")
print(f"  No Failure: {failure_counts.get(0, 0)} ({failure_counts.get(0, 0)/len(df_processed)*100:.1f}%)")
print(f"  Failure: {failure_counts.get(1, 0)} ({failure_counts.get(1, 0)/len(df_processed)*100:.1f}%)")

In [ ]:
# Sensor distributions
sensor_cols = ['engine_temp', 'oil_pressure', 'battery_voltage', 'coolant_level', 
               'transmission_temp', 'brake_pad_wear_avg']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(sensor_cols):
    if col in df_processed.columns:
        # Plot distribution for failure vs no failure
        df_processed[df_processed['failure_within_30_days']==0][col].hist(
            ax=axes[i], bins=30, alpha=0.6, label='No Failure', color='#2ecc71'
        )
        df_processed[df_processed['failure_within_30_days']==1][col].hist(
            ax=axes[i], bins=30, alpha=0.6, label='Failure', color='#e74c3c'
        )
        axes[i].set_title(f'{col.replace("_", " ").title()}', fontsize=12)
        axes[i].legend()

plt.suptitle('Sensor Distributions by Failure Status', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
numeric_cols = ['engine_temp', 'oil_pressure', 'battery_voltage', 'coolant_level',
                'transmission_temp', 'brake_pad_wear_avg', 'odometer', 'rpm',
                'engine_health', 'oil_health', 'battery_health', 'overall_health',
                'risk_score', 'failure_within_30_days']

# Filter to only existing columns
numeric_cols = [col for col in numeric_cols if col in df_processed.columns]

plt.figure(figsize=(14, 10))
corr_matrix = df_processed[numeric_cols].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r',
            center=0, square=True, linewidths=0.5)
plt.title('Feature Correlation Heatmap', fontsize=14)
plt.tight_layout()
plt.show()

# Top correlations with target
if 'failure_within_30_days' in corr_matrix.columns:
    target_corr = corr_matrix['failure_within_30_days'].drop('failure_within_30_days').sort_values(key=abs, ascending=False)
    print("Top correlations with failure:")
    print(target_corr.head(10))

## 5. Prepare Data for Training

In [ ]:
# Select features for training
feature_columns = [
    'engine_temp', 'oil_pressure', 'battery_voltage', 'fuel_level',
    'tire_pressure_avg', 'brake_pad_wear_avg', 'odometer', 'speed',
    'rpm', 'coolant_level', 'transmission_temp', 'check_engine_light',
    'engine_health', 'oil_health', 'battery_health', 'cooling_health',
    'trans_health', 'overall_health', 'mileage_normalized', 'high_mileage',
    'risk_score'
]

# Filter to only existing columns
feature_columns = [col for col in feature_columns if col in df_processed.columns]

print(f"Features selected: {len(feature_columns)}")
print(feature_columns)

# Prepare X and y
X = df_processed[feature_columns].values
y = df_processed['failure_within_30_days'].values

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

In [ ]:
# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining class distribution:")
print(f"  No Failure: {sum(y_train==0)} ({sum(y_train==0)/len(y_train)*100:.1f}%)")
print(f"  Failure: {sum(y_train==1)} ({sum(y_train==1)/len(y_train)*100:.1f}%)")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nFeatures scaled using StandardScaler")

## 6. Train RandomForest Classifier

In [ ]:
# Train RandomForest with hyperparameter tuning
print("Training RandomForest Classifier...")

# Define parameter grid for GridSearchCV
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'class_weight': ['balanced', None]
}

# Initialize model
rf_model = RandomForestClassifier(random_state=42)

# GridSearchCV
grid_search_rf = GridSearchCV(
    rf_model, param_grid_rf, cv=5, scoring='f1',
    n_jobs=-1, verbose=1
)

grid_search_rf.fit(X_train_scaled, y_train)

# Best model
best_rf = grid_search_rf.best_estimator_
print(f"\nBest RandomForest Parameters: {grid_search_rf.best_params_}")
print(f"Best CV F1 Score: {grid_search_rf.best_score_:.4f}")

## 7. Train XGBoost/GradientBoosting Model

In [ ]:
# Train XGBoost or GradientBoosting
print("Training Gradient Boosting Classifier...")

if XGBOOST_AVAILABLE:
    # Calculate scale_pos_weight for imbalanced data
    scale_pos_weight = sum(y_train == 0) / sum(y_train == 1) if sum(y_train == 1) > 0 else 1
    
    param_grid_xgb = {
        'n_estimators': [50, 100],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
    }
    
    xgb_model = XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    
    grid_search_xgb = GridSearchCV(
        xgb_model, param_grid_xgb, cv=5, scoring='f1',
        n_jobs=-1, verbose=1
    )
else:
    # Use GradientBoosting as alternative
    param_grid_gb = {
        'n_estimators': [50, 100],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
    }
    
    gb_model = GradientBoostingClassifier(random_state=42)
    
    grid_search_xgb = GridSearchCV(
        gb_model, param_grid_gb, cv=5, scoring='f1',
        n_jobs=-1, verbose=1
    )

grid_search_xgb.fit(X_train_scaled, y_train)

best_xgb = grid_search_xgb.best_estimator_
print(f"\nBest Boosting Parameters: {grid_search_xgb.best_params_}")
print(f"Best CV F1 Score: {grid_search_xgb.best_score_:.4f}")

## 8. Model Evaluation and Comparison

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    """Evaluate model and return metrics"""
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_test, y_proba) if len(np.unique(y_test)) > 1 else 0
    }
    
    print(f"\n{model_name} Results:")
    print(f"  Accuracy:  {metrics['accuracy']:.4f}")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
    print(f"  ROC-AUC:   {metrics['roc_auc']:.4f}")
    
    return metrics, y_pred, y_proba

# Evaluate both models
print("=" * 60)
print("MODEL EVALUATION ON TEST SET")
print("=" * 60)

rf_metrics, rf_pred, rf_proba = evaluate_model(best_rf, X_test_scaled, y_test, "RandomForest")
xgb_metrics, xgb_pred, xgb_proba = evaluate_model(best_xgb, X_test_scaled, y_test, "GradientBoosting/XGBoost")

# Select best model
if rf_metrics['f1'] >= xgb_metrics['f1']:
    best_model = best_rf
    best_model_name = "RandomForest"
    best_metrics = rf_metrics
else:
    best_model = best_xgb
    best_model_name = "GradientBoosting/XGBoost"
    best_metrics = xgb_metrics

print(f"\n🏆 Best Model: {best_model_name} (F1: {best_metrics['f1']:.4f})")

In [ ]:
# Confusion matrices and ROC curves
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# RandomForest Confusion Matrix
cm_rf = confusion_matrix(y_test, rf_pred)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
axes[0, 0].set_title('RandomForest Confusion Matrix')
axes[0, 0].set_xlabel('Predicted')
axes[0, 0].set_ylabel('Actual')

# GradientBoosting Confusion Matrix
cm_xgb = confusion_matrix(y_test, xgb_pred)
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Greens', ax=axes[0, 1])
axes[0, 1].set_title('GradientBoosting Confusion Matrix')
axes[0, 1].set_xlabel('Predicted')
axes[0, 1].set_ylabel('Actual')

# ROC Curves
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_proba)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_proba)

axes[1, 0].plot(fpr_rf, tpr_rf, label=f'RandomForest (AUC={rf_metrics["roc_auc"]:.3f})', color='blue')
axes[1, 0].plot(fpr_xgb, tpr_xgb, label=f'GradientBoosting (AUC={xgb_metrics["roc_auc"]:.3f})', color='green')
axes[1, 0].plot([0, 1], [0, 1], 'k--', label='Random')
axes[1, 0].set_xlabel('False Positive Rate')
axes[1, 0].set_ylabel('True Positive Rate')
axes[1, 0].set_title('ROC Curves Comparison')
axes[1, 0].legend()

# Model Comparison Bar Chart
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC-AUC']
rf_values = [rf_metrics['accuracy'], rf_metrics['precision'], rf_metrics['recall'], rf_metrics['f1'], rf_metrics['roc_auc']]
xgb_values = [xgb_metrics['accuracy'], xgb_metrics['precision'], xgb_metrics['recall'], xgb_metrics['f1'], xgb_metrics['roc_auc']]

x = np.arange(len(metrics_names))
width = 0.35

axes[1, 1].bar(x - width/2, rf_values, width, label='RandomForest', color='#3498db')
axes[1, 1].bar(x + width/2, xgb_values, width, label='GradientBoosting', color='#2ecc71')
axes[1, 1].set_xlabel('Metric')
axes[1, 1].set_ylabel('Score')
axes[1, 1].set_title('Model Comparison')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(metrics_names)
axes[1, 1].legend()
axes[1, 1].set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

## 9. Feature Importance Analysis

In [ ]:
# Feature importance from best model
importances = best_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': feature_columns,
    'importance': importances
}).sort_values('importance', ascending=True)

# Plot feature importance
plt.figure(figsize=(10, 8))
plt.barh(feature_importance_df['feature'], feature_importance_df['importance'], color='#3498db')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title(f'Feature Importance - {best_model_name}')
plt.tight_layout()
plt.show()

# Top 10 most important features
print("\nTop 10 Most Important Features:")
for i, row in feature_importance_df.tail(10).iloc[::-1].iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

## 10. Save Model Artifacts

In [ ]:
# Create models directory
MODELS_DIR = "../models"
os.makedirs(MODELS_DIR, exist_ok=True)

# Save model and scaler
model_path = os.path.join(MODELS_DIR, "failure_predictor.joblib")
scaler_path = os.path.join(MODELS_DIR, "scaler.joblib")

joblib.dump(best_model, model_path)
joblib.dump(scaler, scaler_path)

# Save feature list
feature_list_path = os.path.join(MODELS_DIR, "features.json")
with open(feature_list_path, 'w') as f:
    json.dump(feature_columns, f, indent=2)

# Save model metadata
metadata = {
    'model_type': best_model_name,
    'training_date': datetime.now().isoformat(),
    'n_samples': len(X),
    'n_features': len(feature_columns),
    'features': feature_columns,
    'metrics': {
        'accuracy': float(best_metrics['accuracy']),
        'precision': float(best_metrics['precision']),
        'recall': float(best_metrics['recall']),
        'f1_score': float(best_metrics['f1']),
        'roc_auc': float(best_metrics['roc_auc'])
    },
    'hyperparameters': grid_search_rf.best_params_ if best_model_name == "RandomForest" else grid_search_xgb.best_params_,
    'version': '1.0.0'
}

metadata_path = os.path.join(MODELS_DIR, "model_metadata.json")
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Model artifacts saved:")
print(f"   - Model: {model_path}")
print(f"   - Scaler: {scaler_path}")
print(f"   - Features: {feature_list_path}")
print(f"   - Metadata: {metadata_path}")

## 11. Test Prediction API Response Format

In [ ]:
def predict_failure(vehicle_data, model, scaler, feature_cols):
    """
    Generate prediction in API response format.
    
    Returns:
        dict: {probability, failure_window_days, severity, component, explanation}
    """
    # Prepare features (assuming vehicle_data is a dict)
    features = np.array([[vehicle_data.get(col, 0) for col in feature_cols]])
    features_scaled = scaler.transform(features)
    
    # Get prediction
    probability = model.predict_proba(features_scaled)[0][1]
    
    # Determine severity
    if probability > 0.8:
        severity = "Critical"
        failure_window = max(1, int(7 * (1 - probability)))
    elif probability > 0.6:
        severity = "High"
        failure_window = max(7, int(14 * (1 - probability)))
    elif probability > 0.4:
        severity = "Medium"
        failure_window = max(14, int(30 * (1 - probability)))
    elif probability > 0.2:
        severity = "Low"
        failure_window = max(30, int(60 * (1 - probability)))
    else:
        severity = "Normal"
        failure_window = 90
    
    # Identify potential issues
    issues = []
    if vehicle_data.get('engine_temp', 90) > 105:
        issues.append("Engine overheating")
    if vehicle_data.get('oil_pressure', 40) < 30:
        issues.append("Low oil pressure")
    if vehicle_data.get('battery_voltage', 12) < 11.5:
        issues.append("Battery degradation")
    if vehicle_data.get('brake_pad_wear_avg', 0.5) < 0.2:
        issues.append("Brake pads need replacement")
    if vehicle_data.get('coolant_level', 1) < 0.6:
        issues.append("Low coolant")
    
    # Generate explanation
    if issues:
        explanation = f"Detected issues: {', '.join(issues)}. "
    else:
        explanation = "No critical issues detected. "
    
    if severity in ["Critical", "High"]:
        explanation += "Immediate service recommended."
    elif severity == "Medium":
        explanation += "Schedule service within 1-2 weeks."
    else:
        explanation += "Continue regular maintenance schedule."
    
    return {
        "vehicle_id": vehicle_data.get('vehicle_id', 'UNKNOWN'),
        "failure_probability": round(float(probability), 4),
        "failure_window_days": failure_window,
        "severity": severity,
        "predicted_failure_types": issues if issues else ["No immediate concerns"],
        "explanation": explanation,
        "confidence": 0.85,
        "timestamp": datetime.now().isoformat()
    }

# Test with sample vehicle data
test_vehicle = {
    'vehicle_id': 'VH007',
    'engine_temp': 120.2,
    'oil_pressure': 25.5,
    'battery_voltage': 10.8,
    'fuel_level': 12.5,
    'tire_pressure_avg': 25.1,
    'brake_pad_wear_avg': 0.11,
    'odometer': 156780,
    'speed': 42,
    'rpm': 2100,
    'coolant_level': 0.42,
    'transmission_temp': 125.5,
    'check_engine_light': 1,
    'engine_health': 0.25,
    'oil_health': 0.57,
    'battery_health': 0.27,
    'cooling_health': 0.42,
    'trans_health': 0.09,
    'overall_health': 0.32,
    'mileage_normalized': 0.78,
    'high_mileage': 1,
    'risk_score': 4
}

prediction = predict_failure(test_vehicle, best_model, scaler, feature_columns)
print("API Response Format Test:")
print(json.dumps(prediction, indent=2))

## Summary

### Model Performance
- **Best Model**: Selected based on F1-score to balance precision and recall
- **Key Features**: Engine temperature, oil pressure, overall health score, and risk indicators
- **Use Case**: Real-time failure prediction for vehicle maintenance scheduling

### Next Steps
1. Deploy model via FastAPI endpoint (`/predict`)
2. Set up model retraining pipeline
3. Monitor model drift and performance
4. Integrate with Master Agent for automated scheduling

### Model Artifacts Location
- `../models/failure_predictor.joblib` - Trained model
- `../models/scaler.joblib` - Feature scaler
- `../models/features.json` - Feature list
- `../models/model_metadata.json` - Training metadata